In [ ]:
# https://www.kaggle.com/c/santander-product-recommendation/overview

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
sample_submission = pd.read_csv("./sample_submission.csv")

c:\users\administrator\basic\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\administrator\basic\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
sample_submission.tail()

,ncodpers,added_products
929610,1553685,ind_tjcr_fin_ult1
929611,1553686,ind_tjcr_fin_ult1
929612,1553687,ind_tjcr_fin_ult1
929613,1553688,ind_tjcr_fin_ult1
929614,1553689,ind_tjcr_fin_ult1


#### column descriptions

- fecha_dato : the table is partitioned for this column(해당 데이터를 만든? 날짜라고 생각하면 될듯..)


- ncodpers : customer code (고객 코드, 코드-고객 1:1매칭.. 중복되는 코드가 있을 수 있다)


- ind_empleado : employee index (A active, B ex employed, F filial, N not employee, P pasive)


- pais_residencia : customer's country residence (고객 국적)


- sexo : customers's sex(고객 성별)


- age : age(고객 나이)


-  fecha_alta : the date in which the customer became as the first holder of a contract in the bank
(계약 시작 날짜로 생각하면 될듯)


- ind_nuevo : new customer index. 1 if the customer registered in the last 6 months(최근 6개월 내 계약한 고객)


- antiguedad : customer seniority(in months) fecha_alta와 fecha_dato사이 간격(개월 수)


- indrel : 1(First/primary), 99(primary customer during the month but not at the end of the month)


- ult_fec_cli_1t : last date as primary customer(if he isn't at the end of the month)


- indrel_1mes : customer type at the beginning of the month, 1 (First/Primary customer), 2 (co-owner),P (Potential),3 (former primary), 4(former co-owner)


- tiprel_1mes : customer relation type at the beginning of the month.  A (active), I (inactive), P (former customer),R (Potential)


- indresi : residence index (S (Yes) or N (No) if the residence country is the same than the bank country)


- indext : Foreigner index (S (Yes) or N (No) if the customer's birth country is different than the bank country)


- conyuemp : spouse index. 1 if the customer is spouse of an employee


- canal_entrada : channel used by the customer to join


- indfall : deceased index. N/S


- tipodom : address type. 1, primary address


- cod-prov : province cod(customer's address)


- nomprov : province name


- ind_actividad_cliente : activity index(1 : active customer, 0 : inactive customer)


- renta : gross income of the household


- segmento : segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

In [3]:
train.tail()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
13647304,2016-05-28,1166765,N,ES,V,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647305,2016-05-28,1166764,N,ES,V,23,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647306,2016-05-28,1166763,N,ES,H,47,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647307,2016-05-28,1166789,N,ES,H,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647308,2016-05-28,1550586,N,ES,H,37,2016-05-13,1.0,0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [10]:
train.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [4]:
test.tail()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
929610,2016-06-28,660237,N,ES,V,55,1999-04-21,0,206,1,...,N,NaN,KAT,N,1,28.0,MADRID,1,128643.57,01 - TOP
929611,2016-06-28,660238,N,ES,V,30,2006-11-29,0,115,1,...,N,NaN,KFC,N,1,26.0,"RIOJA, LA",0,NA,02 - PARTICULARES
929612,2016-06-28,660240,N,ES,V,52,2006-11-29,0,115,1,...,N,NaN,KBZ,N,1,33.0,ASTURIAS,1,72765.27,02 - PARTICULARES
929613,2016-06-28,660243,N,ES,V,32,2006-11-29,0,115,1,...,N,NaN,KFC,N,1,33.0,ASTURIAS,0,147488.88,02 - PARTICULARES
929614,2016-06-28,660248,N,ES,V,92,2006-11-29,0,115,1,...,N,NaN,KAT,N,1,45.0,TOLEDO,0,NA,02 - PARTICULARES
